In [1]:
import sys
import csv
from pyspark.sql import SparkSession
from pyspark.sql.functions import format_string
from pyspark.sql.functions  import date_format
spark = SparkSession.builder.appName("app").config("spark.some.config.option", "some-value").getOrCreate()

In [2]:
#data =spark.read.format('csv').options(header='true',inferschema='true').load("/FileStore/tables/testFile.csv")
data =spark.read.format('csv').options(header='true',inferschema='true').option("timestamp", "MM/dd/yyyy HH:mm:ss").load("/FileStore/tables/data.csv")

In [3]:
data= data.withColumnRenamed('Primary Type','PType')
data= data.withColumnRenamed('Case Number','CNum')
data= data.withColumnRenamed('Location Description','LocDes')
data= data.withColumnRenamed('Community Area','CommArea')
data= data.withColumnRenamed('FBI Code','FBICode')
data= data.withColumnRenamed('X Coordinate','Xcoord')
data= data.withColumnRenamed('Y Coordinate','Ycoord')
data= data.withColumnRenamed('Updated On','UpdatedOn')

In [4]:
data.createOrReplaceTempView('data')

In [5]:
groupByDate = spark.sql('select YEAR(Date) as year,count(*) as cnt from data group by YEAR(Date)').collect()

In [6]:
for row in groupByDate:
  print('{0},{1}'.format(row['year'],row['cnt']))

In [7]:
spark.sql('select YEAR(Date),count(*) from data group by YEAR(Date)').collect()

In [8]:
data.createOrReplaceTempView('data')

In [9]:
distCdTpDes = spark.sql('select distinct Iucr,PType,Description from data order by Iucr').collect()
for row in distCdTpDes:
  print('{0},{1},{2}'.format(row['Iucr'],row['PType'],row['Description']))

In [10]:
distCdTpDes= spark.sql('select Iucr,PType,Description,count(*) as co from data group by Iucr,PType,Description order by Iucr').collect()
for row in distCdTpDes:
  print('{0},{1},{2},{3}'.format(row['Iucr'],row['PType'],row['Description'],row['co']))

In [11]:
groupByDay = spark.sql('select DAY(Date) as days,count(*) as cnt from data group by DAY(Date)').collect()
for row in groupByDay:
  print('{0},{1}'.format(row['days'],row['cnt']))

In [12]:
groupByMonth = spark.sql('select MONTH(Date) as mon,count(*) as cnt from data group by MONTH(Date)').collect()
for row in groupByMonth:
  print('{0},{1}'.format(row['mon'],row['cnt']))

In [13]:
countPerDist = spark.sql('select District,count(*) as distCnt from data group by District').collect()
for row in countPerDist:
  print('{0},{1}'.format(row['District'], row['distCnt']))

In [14]:
countPerDistPerYr = spark.sql('select District,YEAR(Date) as yr,count(*) as distCnt from data group by District,YEAR(Date)').collect()
for row in countPerDistPerYr:
  print('{0},{1},{2}'.format(row['District'], row['yr'],row['distCnt']))

In [15]:
arrestPerOffense = spark.sql('select PType,count(*) as arrestCount from data where Arrest = TRUE group by PType order by PType').collect()
for row in arrestPerOffense:
  print('{0},{1}'.format(row['PType'],row['arrestCount']))

In [16]:
#data.columns
spark.sql('select distinct Description from data').collect()

In [17]:
crimePerYear = spark.sql('select YEAR(Date) as Year,PType,count(*) as crimecount from data group by PType,YEAR(Date) order by PType').collect()
for row in crimePerYear:
  print('{0},{1},{2}'.format(row['Year'],row['PType'],row['crimecount']))

In [18]:
sex_assault = spark.sql('select count(*) as sex_assault_cnt, YEAR(Date) as year,Description from data where PType = "CRIM SEXUAL ASSAULT" group by YEAR(Date),Description').collect()

for row in sex_assault:
  print('{0},{1},{2}'.format(row['sex_assault_cnt'],row['year'],row['Description']))

In [19]:
#chceking duplicate counts
#spark.sql('SELECT ID, COUNT(ID) FROM data GROUP BY ID HAVING COUNT(ID)>1').collect()
import csv
dic = {}
check_block = spark.sql('select Block,count(*) as location_count from data where Latitude is NULL group by Block').collect()
for row in check_block:
  dic[row['Block']] = row['location_count']

with open('mycsvfile.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, dic.keys())
    w.writeheader()
    mycsvfile.writerow(dic)

In [20]:
spark.sql('select distinct PType,IUCR from data').collect()